In [20]:
import torch
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pysam
import yfinance as yf

In [21]:
'''
data = yf.download(
    "USD/PLN",
    start="2020-01-01",
    end="2024-01-01", auto_adjust= True
)
data'''

'\ndata = yf.download(\n    "USD/PLN",\n    start="2020-01-01",\n    end="2024-01-01", auto_adjust= True\n)\ndata'

In [32]:

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("GPU dla M1 pro")

GPU dla M1 pro


In [22]:
dataset1=pd.read_csv("USD_PLN1.csv",index_col=False)
dataset2=pd.read_csv("USD_PLN2.csv",index_col=False)
dataset=pd.concat([dataset1,dataset2],ignore_index=True)

dataset["Date"] = pd.to_datetime(dataset["Date"])
dataset=dataset.sort_values(by="Date", ascending=True)
dataset = dataset.set_index("Date")

dataset["Change %"] = dataset["Change %"].str.replace('%','').astype(float)/100
dataset = dataset.drop(columns=['Vol.'])

In [23]:
type(dataset["Change %"].iloc[0])


numpy.float64

In [24]:
dataset

,Price,Open,High,Low,Change %
Date,,,,,
1997-10-01,3.4275,3.4150,3.4298,3.4090,0.0037
1997-10-02,3.4210,3.4290,3.4330,3.4193,-0.0019
1997-10-03,3.4120,3.4170,3.4373,3.4135,-0.0026
1997-10-06,3.4185,3.4135,3.4618,3.4100,0.0019
1997-10-07,3.4100,3.4183,3.4203,3.4050,-0.0025
...,...,...,...,...,...
2025-10-30,3.6709,3.6565,3.6787,3.6451,0.0039
2025-10-31,3.6875,3.6709,3.6972,3.6636,0.0045
2025-11-02,3.6920,3.6821,3.6925,3.6886,0.0012


In [50]:
class waluty(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,output_size=1,dropout_prob=0.4):
        super().__init__()

        self.hidden_size=hidden_size
        self.num_layers= num_layers
       
        #nienawidze zimy
        self.lstm=nn.LSTM(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True, # to tylko zmiena kolejnosc w przyjmowaniudanych ale szczegółów nie znam
                          dropout=dropout_prob if num_layers>1 else 0 )

        self.classifier=nn.Sequential(
            nn.Linear(hidden_size,32),
            nn.Dropout(dropout_prob),
            nn.ReLU(),
            nn.Linear(32,output_size))

        '''
        x.shape = (batch_size, seq_len, input_size)

        lstm_out, hidden = self.lstm(x)

        1)
        lstm_out.shape = (batch_size, seq_len, hidden_size)
        #stany ukryte dla kazdego kroku
        czyli
        lstm_out[b, t] = stan LSTM po kroku t
        to stan LSTM po kroku numer t




        2)
        h_n.shape = (num_layers, batch_size, hidden_size)
        c_n.shape = (num_layers, batch_size, hidden_size)


        h_n → ostatni hidden state
        c_n → ostatni cell state
        tylko OSTATNI krok czasowy
        osobno dla każdej warstwy




        my wybieramy ostatni krok z lstm_out bo ostatni hidden state = podsumowanie sekwencji
        '''




    
    def forward(self,x):
        lstm_out, hidden=self.lstm(x)
        last_step = lstm_out[:, -1, :]
        classifier=self.classifier(last_step)
        return classifier

        
        
        

In [51]:
input_size=5
hidden_size=64
num_layers=2

model=waluty(input_size,hidden_size,num_layers).to(device)
print(model)

waluty(
  (lstm): LSTM(5, 64, num_layers=2, batch_first=True, dropout=0.4)
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [52]:
dane_testowe = torch.randn(1, 4, input_size).to(device)

dane_testowe.shape
model(dane_testowe)




#randn(batch_size, seq_len, features)

#batch_size = ile oddzielnych próbek wrzucasz do sieci jednocześnie
#seq_len = ile dni do tyłu patrzymy(60 to 60 dni do tyłu)
#features = ile cech (liczb) opisuje (ile mamy wejsciowych kolumn)



tensor([[-0.0645]], device='mps:0', grad_fn=<LinearBackward0>)